In [12]:
import ipywidgets as widgets
from ipywidgets import interact
from src.post_processing import PathWrangler
from IPython.display import display
import pandas as pd
from ipysheet import from_dataframe

In [13]:
path_filepath = '../artifacts/processed_expansions/found_paths.json'
predicted_reactions_filepath = "../artifacts/processed_expansions/predicted_reactions.json"
known_reactions_filepath = "../artifacts/processed_expansions/known_reactions.json"

pw = PathWrangler(
    path_filepath=path_filepath,
    pr_filepath=predicted_reactions_filepath,
    kr_filepath=known_reactions_filepath
)

In [31]:
starter_options, target_options = zip(*pw.starter_targets)
target_options = tuple(set(target_options))
starter_selection = list(starter_options)
target_selection = list(target_options)

starter_selector = widgets.SelectMultiple(
    options=starter_options,
    value=starter_options,
    rows=len(starter_options),
    description='Starters:',
    disabled=False
)

def starter_listener(change):
    global starter_selection
    starter_selection = starter_selector.value

starter_selector.observe(starter_listener)

target_selector = widgets.SelectMultiple(
    options=target_options,
    value=target_options,
    rows=len(target_options),
    description='Targets:',
    disabled=False
)

def target_listener(change):
    global target_selection
    target_selection = target_selector.value

target_selector.observe(target_listener)

evidence_options = {elt.value: elt.name for elt in pw.enzyme_existence}
default_loe = pw.enzyme_existence.PROTEIN.value
loe_selection = [evidence_options[default_loe]]

evidence_selector = widgets.SelectMultiple(
    options=evidence_options.keys(),
    value=[default_loe],
    rows=len(evidence_options),
    description='Enzyme level of evidence:',
    disabled=False
)

def evidence_listener(change):
    global loe_selection
    loe_selection = [evidence_options[k] for k in evidence_selector.value]

evidence_selector.observe(evidence_listener)

sort_by_selection = 'mean_rcmcs'
sort_options = [
    ("Mean RCMCS", 'mean_rcmcs'),
    ("Min RCMCS", 'min_rcmcs'),
    ("Max-min driving force", 'mdf')
]

sort_by_radio_buttons = widgets.RadioButtons(
    options=sort_options,
    value=sort_by_selection,
    description='Sort paths by:',
    disabled=False
)

def sort_by_listener(change):
    global sort_by_selection
    sort_by_selection = sort_by_radio_buttons.value

sort_by_radio_buttons.observe(sort_by_listener)

paths = []
button_out = widgets.Output()
def click_handler(b):
    global paths
    filter_by_enzymes = {'existence': loe_selection} # Only paths w/ enzymes w/ protein-level evidence
    paths = pw.get_paths(
        starters=starter_selection,
        targets=target_selection,
        sort_by=sort_by_selection,
        filter_by_enzymes=filter_by_enzymes
    )
    with button_out:
        print(f"{len(paths)} paths meet your criteria")

get_paths_buttton = widgets.Button(
    description='Get paths',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='solid flask' # (FontAwesome names without the `fa-` prefix)
)

get_paths_buttton.on_click(click_handler)

In [32]:
selectors = widgets.HBox(
    [
        starter_selector,
        target_selector,
        evidence_selector,
        sort_by_radio_buttons
    ]
)

interface = widgets.VBox([selectors, get_paths_buttton])
display(interface)
display(button_out)

Output()

Predicted pathway viewer

In [64]:
path_selector = widgets.Dropdown(
    options=[(i+1, i) for i in range(len(paths))],
    value=0,
    description='Path #',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def display_enzymes(enzymes):
    df = pd.DataFrame(data=[e.to_dict() for e in enzymes])
    df = df.loc[:, ['uniprot_id', 'ec', 'organism', 'name', 'existence', 'reviewed', 'sequence']]

    return from_dataframe(df)

def display_predicted_reaction(rxn_step, img):
    html = f"""
    <b><u>Step #{rxn_step+1}</u></b>
    <div>
        <img src="{img}" style="width:950px">
    </div>
    """
    return widgets.HTML(html)

def display_analogue(img, rcmcs):
    html = f"""
    <b><u>{rcmcs * 100:.2f}% similar to predicted reaction</u></b>
    <div>
        <img src="{img}" style="width:900px">
    </div>
    """
    return widgets.HTML(value=html)

def display_analogues_enzymes(krs, rcmcses):
    kr_elts = []
    enzyme_elts = []
    for kr, rcmcs in zip(krs, rcmcses):
        kr_elts.append(display_analogue(kr.image, rcmcs))
        enzyme_elts.append(display_enzymes(kr.enzymes))

    kr_selector = widgets.Dropdown(
        options=[(i+1, i) for i in range(len(kr_elts))],
        value=0,
        description="Known analogue: "
    )
    kr_stack = widgets.Stack(kr_elts, selected_index=0)
    kr_sel_disp = widgets.VBox([kr_selector, kr_stack])
    enzyme_stack = widgets.Stack(enzyme_elts, selected_index=0)
    link_kr = widgets.jslink((kr_selector, 'index'), (kr_stack, 'selected_index'))
    link_enz = widgets.jslink((kr_selector, 'index'), (enzyme_stack, 'selected_index'))

    tab_titles = ['Known Analogue', 'Enzymes']
    tab_children = [kr_sel_disp, enzyme_stack]
    tab = widgets.Tab(
        children=tab_children,
        titles=tab_titles,
        layout=widgets.Layout(width="950px")
    )

    return tab

@interact
def display_path(path_idx=path_selector, k=3):
    path = paths[path_idx]
    data = (
        (
            reaction.image,
            reaction.top_rcmcs(k),
            reaction.top_analogues(k=k)
        )
        for reaction in path.reactions
    )
    header = f"""
    <h3>{len(path.reactions)}-step path from {path.starter.upper()} to {path.target.upper()}<br>
    Max-min driving force = {path.mdf:.2f} kJ/mol<br>
    </h3>
    """
    rows = [widgets.HTML(value=header)]
    for i, (pr_image, rcmcses, analogues) in enumerate(data):
        pr_elt = display_predicted_reaction(i, pr_image)
        kr_elt = display_analogues_enzymes(analogues, rcmcses)
        rows.append(widgets.HBox([pr_elt, kr_elt], layout=widgets.Layout(border= '1px solid black')))

    return widgets.VBox(rows)

interactive(children=(Dropdown(description='Path #', options=((1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), …